In [2]:
# Imports
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG, clear_output, HTML
import matplotlib.pyplot as plt

from pydrake.all import *
from pydrake.examples.manipulation_station import ManipulationStation

from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import (
    StartMeshcat, MeshcatJointSlidersThatPublish)
from manipulation.scenarios import AddMultibodyTriad, SetColor

import pandas as pd
import joblib
import sys
import json

In [3]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
# urdf_path = './URDFModelTang/urdf/URDFModelTang.urdf'
urdf_path = "./URDF_Model_22-02-28/urdf/URDF_Model_22-02-28.urdf"
def gripper_forward_kinematics_example():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'diva_robot')
    # Transform for the robot location
    X_R = RigidTransform(RotationMatrix.MakeYRotation(0), np.array([0, 0.5, -1.5]))
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), X_R)
    plant.Finalize()
    
    end_frame = "Sixthlink"
    # Draw the frames
    for body_name in ["Sixthlink"]:
#     for body_name in ["base_link", end_frame, "Fifthlink", "Sixthlink"]:
        AddMultibodyTriad(plant.GetFrameByName(body_name, model), scene_graph)
    
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    
    gripper = plant.GetBodyByName(end_frame)
    def pose_callback(context):
        pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
        print(pose.translation())
        clear_output(wait=True)
        print("gripper position (m): " + np.array2string(
            pose.translation(), formatter={
                'float': lambda x: "{:3.2f}".format(x)}))
        print("gripper roll-pitch-yaw (rad):" + np.array2string(
            RollPitchYaw(pose.rotation()).vector(),
                         formatter={'float': lambda x: "{:3.2f}".format(x)}))
        print("pose rotation: ", pose.rotation())
    sliders = MeshcatJointSlidersThatPublish(meshcat, plant, visualizer, context)
    # sliders.Run()
    sliders.Run(pose_callback)

gripper_forward_kinematics_example()

gripper position (m): [-0.83 -2.83 -1.29]
gripper roll-pitch-yaw (rad):[1.19 0.40 0.58]
pose rotation:  RotationMatrix([
  [0.7711202255820683, 0.09828163495643506, 0.6290582786424087],
  [0.5036740131086793, 0.510212718967761, -0.6971337532514995],
  [-0.38946897978819606, 0.8544142447512298, 0.34393344145187027],
])
[-0.8314602  -2.82762998 -1.29048343]
